In [6]:
import numpy as np
import pandas as pd
import os

new_dir = r"E:/vit-yolo-final"
os.chdir(new_dir)
print("Current Directory:", os.getcwd())

Current Directory: E:\vit-yolo-final


In [8]:
!pip install --upgrade ultralytics timm torch


  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)


In [9]:
!pip install transformers


In [10]:
# hybrid_vit_yolo_fixed.py
import torch
import torch.nn as nn
from torchvision import transforms
from transformers import ViTModel, ViTFeatureExtractor
from ultralytics import YOLO
from PIL import Image

# ViT Wrapper to simulate CNN-like backbone
class ViTBackbone(nn.Module):
    def __init__(self, model_name='google/vit-base-patch16-384'):
        super().__init__()
        self.vit = ViTModel.from_pretrained(model_name)
        self.proj = nn.Conv2d(768, 256, kernel_size=1)

    def forward(self, x):
        # ViT expects input size (B, 3, 384, 384)
        B, C, H, W = x.shape
        x = self._resize_with_padding(x, 384)
        outputs = self.vit(pixel_values=x, output_hidden_states=True)
        features = outputs.last_hidden_state[:, 1:]  # Remove CLS token
        B, N, D = features.shape
        h = w = int(N ** 0.5)
        x = features.permute(0, 2, 1).view(B, D, h, w)
        return self.proj(x)

    def _resize_with_padding(self, x, target_size):
        resize = transforms.Resize((target_size, target_size))
        return resize(x)

# ViT + YOLO Hybrid Model
class ViTYOLO(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        self.backbone = ViTBackbone()
        self.yolo = YOLO('yolov8n.yaml')
        self.yolo.model.model[0] = self.backbone  # Replace backbone

    def forward(self, x):
        return self.yolo(x)

# Training function
def train():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = ViTYOLO().to(device)

    model.yolo.train(
        data='your_dataset.yaml',
        imgsz=384,  # Required for ViT input
        epochs=100,
        batch=8,
        optimizer='AdamW',
        lr0=1e-4
    )

if __name__ == "__main__":
    train()


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-384 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=your_dataset.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=384, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simpl

100%|█████████████████████████████████████████████████████████████████████████████| 5.35M/5.35M [00:01<00:00, 4.13MB/s]


AMP: checks passed 


train: Scanning E:\vit-yolo-final\labels\train... 5984 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5984/5984 [00


...

                   all       1497       8231      0.458      0.391      0.387      0.206

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100     0.877G      1.392     0.9466      1.033         51        384: 100%|██████████| 748/748 [03:13<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [00:26


                   all       1497       8231      0.469      0.389      0.387      0.206

100 epochs completed in 6.073 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.109  Python-3.12.4 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLOv8n summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [00:30


                   all       1497       8231      0.477      0.403      0.396      0.213
                   car       1338       5889      0.634      0.746      0.764      0.472
            pedestrian        359        916      0.467      0.395      0.388      0.174
                   van        427        570      0.553      0.376      0.429      0.257
               cyclist        222        308      0.433      0.195      0.198     0.0851
                 truck        220        227      0.652      0.577      0.627      0.359
                  misc        156        187      0.424      0.176      0.195     0.0979
                  tram         62         95      0.362      0.526       0.46      0.229
        person_sitting         20         39      0.289      0.229      0.105     0.0304
Speed: 0.3ms preprocess, 3.1ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs\detect\train


In [14]:
metrics = model.val()

# Print precision, recall, mAP@50, and mAP@50-95
print(f"Precision: {metrics.box.mp:.4f}")  # Mean precision
print(f"Recall: {metrics.box.mr:.4f}")  # Mean recall
print(f"mAP@50: {metrics.box.map50:.4f}")  # Mean AP at IoU 0.5
print(f"mAP@50-95: {metrics.box.map:.4f}")  # Mean AP from 0.5 to 0.95

NameError: name 'model' is not defined